In [1]:
import pandas as pd
from pulp import *

In [2]:
# Importing data
df_origins = pd.read_csv("origins.csv") 
df_destinations = pd.read_csv("destinations.csv")
df_distances = pd.read_csv("distance matrix.csv", index_col=0).transpose()

In [3]:
# Origins, Destinations, demand, and distances
origin = df_origins["City/town"].tolist()
destination = df_destinations["City/town"].tolist()

demand = df_destinations.set_index(df_destinations.columns[0]).to_dict()[df_destinations.columns[1]]
distance = df_distances.to_dict(orient='Distance(Km)')

C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\2214405527.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  distance = df_distances.to_dict(orient='Distance(Km)')
C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\2214405527.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  distance = df_distances.to_dict(orient='Distance(Km)')


In [4]:
print(origin[0], destination[0])
print (demand['Guntur'])
print (distance['Agra'])

PortBlair Visakhapatnam
3020.742036
{'Visakhapatnam': 1592.47051, 'Guntur': 1236.82943, 'Kadapa': 1604.932, 'Kurnool': 1263.067516, 'Guwhati': 1370.940209, 'Goalpara': 1258.659879, 'Goalpara.1': 1258.659879, 'Tezpur': 1467.540004, 'Silchar': 1500.598704, 'Aurangabad': 691.8950419, 'Bhagalpur': 920.0918976, 'ChamparanEast': 1787.610998, 'Darbangha': 792.0115354, 'Mazaffanagar': 48.00884679, 'Patna': 728.0325838, 'Purnia': 955.2268827, 'Saran': 1201.876039, 'Raipur': 757.4165528, 'Bastar': 984.897466, 'Sarguja': 1201.876039, 'Kurukshetra': 328.2412754, 'Gurgaon': 1236.82943, 'Hisar': 312.6855303, 'Rohtak': 235.1273107, 'Shimla': 443.0157105, 'Mandi': 816.7608608, 'Dharamshala': 926.6138013, 'Deoghar': 919.9605259, 'Ranchi': 851.9570984, 'Banglore': 206.557287, 'Chitradurga': 1286.649912, 'Gulbarga': 1102.034188, 'Manglore': 1626.231904, 'Mysore': 1662.375992, 'Belgaum': 1309.951841, 'Bagalkote': 691.8950419, 'Thiruvanantapuram': 110.9987246, 'Eranakulam': 1921.579241, 'Calicut(Khozikode)

In [5]:
# setting up the problem
prob = LpProblem("Transportation", LpMinimize)

In [6]:
# decision variables
routes = [(i, j) for i in origin for j in destination]
ship_qty = LpVariable.dicts("ship", (origin, destination), 0)

In [7]:
# Objective function
prob += lpSum(ship_qty[i][j]*distance[i][j] for (i,j) in routes)

In [8]:
# constraints
for j in destination:
    prob += lpSum(ship_qty[i][j] for i in origin) >= demand[j]  

In [9]:
prob.solve

<bound method LpProblem.solve of Transportation:
MINIMIZE
1382.542847*ship_Agra_Agartala + 343.8039384*ship_Agra_Ajmer + 727.5571647*ship_Agra_Akola + 77.43919537*ship_Agra_Aligarh + 428.9176269*ship_Agra_Allahabad + 428.9176269*ship_Agra_Almora + 578.6923918*ship_Agra_Amritsar + 1383.7900838*ship_Agra_Aurangabad + 691.8950419*ship_Agra_Azamgarh + 691.8950419*ship_Agra_Bagalkote + 312.4268771*ship_Agra_Balasore + 206.557287*ship_Agra_Banglore + 189.9350464*ship_Agra_Bareilly + 984.897466*ship_Agra_Bastar + 473.3059056*ship_Agra_Basti + 1309.951841*ship_Agra_Belgaum + 920.0918976*ship_Agra_Bhagalpur + 50.63804697*ship_Agra_Bharatpur + 441.5080565*ship_Agra_Bhopal + 1106.89223*ship_Agra_Bhubaneswar + 471.1254457*ship_Agra_Bikaner + 706.0108804*ship_Agra_Bolangir + 135.9755859*ship_Agra_Bundelkhand + 1787.610998*ship_Agra_Calicut(Khozikode) + 1787.610998*ship_Agra_ChamparanEast + 1585.5017*ship_Agra_Chennai + 1286.649912*ship_Agra_Chitradurga + 1286.649912*ship_Agra_Chitrakoot + 1802.6610

In [10]:
feasibility_status = prob.solve()

# Print the status
print("Status:", LpStatus[feasibility_status])

# Check for additional information
if LpStatus[feasibility_status] == 'Not Solved':
    print("Solver message:", LpSolverDefault.msg[feasibility_status])

Status: Optimal


In [11]:
solution_df = pd.DataFrame(columns=['origin', 'destination', 'shipment'])
for v in prob.variables():
    if v.varValue > 0:
        _, i, j = v.name.split('_')
        solution_df = solution_df.append({'origin': i, 'destination': j, 'shipment': v.varValue}, ignore_index=True)

solution_df.to_csv('optimal_shipment.csv', index=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\3010096255.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append({'origin': i, 'destination': j, 'shipment': v.varValue}, ignore_index=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\3010096255.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append({'origin': i, 'destination': j, 'shipment': v.varValue}, ignore_index=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\3010096255.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append({'origin': i, 'destination': j, 'shipment': v.varValue}, ignore_index=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8684\3010096255.py:5: FutureWa

In [12]:
print (prob.objective.value())

10991646.673087964
